# Applications of Regular Expressions

**Author:** Bruno Grande

**Date:** August 29th, 2017

## Introduction

In this lesson, we will cover a few applications of regular expressions (or regex) that I use all the time. Regex are available in most programming languages, but to keep this lesson accessible to as many people as possible, we will focus on applications at the Bash shell. Specifically, we will cover how you can use `grep`, `sed` and `awk` to get a lot done without firing up a script, especially with the power of regex at your side. 

### The motivation

Regular expressions are an extremely powerful tool for pattern matching. You might not realize it, but a lot of what we do is pattern matching, especially if you deal with text at all. The ability to describe a flexible pattern that the computer can then quickly look for in some arbitrary text opens up a world of possibilities. This lesson will focus on some of these possibilities. Notably, we will cover:

1. Subsetting text using `grep`
2. Search-and-replace text using `sed`
3. Filter and/or process tabular data using `awk`

These three tools alone justify learning Bash to make your life easier. In combination with regex, they are life-savers! 

### The dataset

We will be using Jenny Bryan's cleaned-up version of the gapminder dataset. It contains 1704 rows and 6 columns. The dataset consists of the population, life expectancy and GDP per capita for 142 countries every 5 years between 1952 and 2007. You can easily download the data using `curl` as follows. 

In [11]:
curl -sL bit.ly/gapm-data > gapminder.tsv

In [12]:
head gapminder.tsv

country	continent	year	lifeExp	pop	gdpPercap
Afghanistan	Asia	1952	28.801	8425333	779.4453145
Afghanistan	Asia	1957	30.332	9240934	820.8530296
Afghanistan	Asia	1962	31.997	10267083	853.10071
Afghanistan	Asia	1967	34.02	11537966	836.1971382
Afghanistan	Asia	1972	36.088	13079460	739.9811058
Afghanistan	Asia	1977	38.438	14880372	786.11336
Afghanistan	Asia	1982	39.854	12881816	978.0114388
Afghanistan	Asia	1987	40.822	13867957	852.3959448
Afghanistan	Asia	1992	41.674	16317921	649.3413952


----

----

----

## Subsetting text using grep

At its simplest, grep can be used to filter lines based on a pattern. We can start with a plain, non-regex pattern. Here, we subset the file to lines that contains the word `Canada`. 

In [14]:
grep Canada gapminder.tsv

Canada	Americas	1952	68.75	14785584	11367.16112
Canada	Americas	1957	69.96	17010154	12489.95006
Canada	Americas	1962	71.3	18985849	13462.48555
Canada	Americas	1967	72.13	20819767	16076.58803
Canada	Americas	1972	72.88	22284500	18970.57086
Canada	Americas	1977	74.21	23796400	22090.88306
Canada	Americas	1982	75.76	25201900	22898.79214
Canada	Americas	1987	76.86	26549700	26626.51503
Canada	Americas	1992	77.95	28523502	26342.88426
Canada	Americas	1997	78.61	30305843	28954.92589
Canada	Americas	2002	79.77	31902268	33328.96507
Canada	Americas	2007	80.653	33390141	36319.23501


You'll notice that the header lines was removed, because it doesn't contain `Canada`. If we want to ensure that this file remains valid, we need to keep the header. There are multiple ways to do this. 

First, you can grep the header and the `Canada` lines separately. 

In [17]:
grep country gapminder.tsv
grep Canada gapminder.tsv

country	continent	year	lifeExp	pop	gdpPercap
Canada	Americas	1952	68.75	14785584	11367.16112
Canada	Americas	1957	69.96	17010154	12489.95006
Canada	Americas	1962	71.3	18985849	13462.48555
Canada	Americas	1967	72.13	20819767	16076.58803
Canada	Americas	1972	72.88	22284500	18970.57086
Canada	Americas	1977	74.21	23796400	22090.88306
Canada	Americas	1982	75.76	25201900	22898.79214
Canada	Americas	1987	76.86	26549700	26626.51503
Canada	Americas	1992	77.95	28523502	26342.88426
Canada	Americas	1997	78.61	30305843	28954.92589
Canada	Americas	2002	79.77	31902268	33328.96507
Canada	Americas	2007	80.653	33390141	36319.23501


However, you will notice that we are repeating ourselves (the `grep` command and the `gapminder.tsv` file name. Ideally, we want to follow the DRY (don't repeat yourself) principle. 

**N.B.** An astute reader will notice that I can extract the header using `head -1`. Indeed, this would work here, but I am familiar with file formats (_e.g._ VCF variant call format) where the header is neither the first line, nor a predictable number of lines into the file. In these cases, `grep` is more general. 

The second approach involves the use of regex. In fact, `grep` stands for "globally search a regular expression and print". However, because there have been multiple versions of regex over the years and we are used to the more modern versions, we will need to use a variant of grep that enables extended regex. You can either use `grep -E` or `egrep`. I will be using the latter. 

Here, we can start using regex by using the `|` operator, which matches what's on the left **or** on the right. Whenever you use regular expressions, it is safer to quote the pattern using single quotes. 

In [21]:
egrep 'country|Canada' gapminder.tsv

country	continent	year	lifeExp	pop	gdpPercap
Canada	Americas	1952	68.75	14785584	11367.16112
Canada	Americas	1957	69.96	17010154	12489.95006
Canada	Americas	1962	71.3	18985849	13462.48555
Canada	Americas	1967	72.13	20819767	16076.58803
Canada	Americas	1972	72.88	22284500	18970.57086
Canada	Americas	1977	74.21	23796400	22090.88306
Canada	Americas	1982	75.76	25201900	22898.79214
Canada	Americas	1987	76.86	26549700	26626.51503
Canada	Americas	1992	77.95	28523502	26342.88426
Canada	Americas	1997	78.61	30305843	28954.92589
Canada	Americas	2002	79.77	31902268	33328.96507
Canada	Americas	2007	80.653	33390141	36319.23501


If we wanted to include the US in our results, it's as simple as adding another `|` operator in our pattern. 

In [26]:
egrep 'country|Canada|United States' gapminder.tsv

country	continent	year	lifeExp	pop	gdpPercap
Canada	Americas	1952	68.75	14785584	11367.16112
Canada	Americas	1957	69.96	17010154	12489.95006
Canada	Americas	1962	71.3	18985849	13462.48555
Canada	Americas	1967	72.13	20819767	16076.58803
Canada	Americas	1972	72.88	22284500	18970.57086
Canada	Americas	1977	74.21	23796400	22090.88306
Canada	Americas	1982	75.76	25201900	22898.79214
Canada	Americas	1987	76.86	26549700	26626.51503
Canada	Americas	1992	77.95	28523502	26342.88426
Canada	Americas	1997	78.61	30305843	28954.92589
Canada	Americas	2002	79.77	31902268	33328.96507
Canada	Americas	2007	80.653	33390141	36319.23501
United States	Americas	1952	68.44	157553000	13990.48208
United States	Americas	1957	69.49	171984000	14847.12712
United States	Americas	1962	70.21	186538000	16173.14586
United States	Americas	1967	70.76	198712000	19530.36557
United States	Americas	1972	71.34	209896000	21806.03594
United States	Americas	1977	73.38	220239000	24072.63213
United States	Americas	1982	74.65	232187835

`grep` can be as flexible as you need it to be. While it may be contrived, let's say we are interested in the data from 1977 for countries whose names start with `S` (and we want to keep the header). As always, there are multiple was of approaching this problem. 

First, we can use UNIX pipes to perform subsequent filters, one for the countries starting with `S` and another for the rows corresponding to 1977. 

In [34]:
egrep '^(country|S)' gapminder.tsv | egrep '1977'

Sao Tome and Principe	Africa	1977	58.55	86796	1737.561657
Saudi Arabia	Asia	1977	58.69	8128505	34167.7626
Senegal	Africa	1977	48.879	5260855	1561.769116
Serbia	Europe	1977	70.3	8686367	12980.66956
Sierra Leone	Africa	1977	36.788	3140897	1348.285159
Singapore	Asia	1967	67.946	1977600	4977.41854
Singapore	Asia	1977	70.795	2325300	11210.08948
Singapore	Asia	2002	78.77	4197776	36023.1054
Slovak Republic	Europe	1977	70.45	4827803	10922.66404
Slovenia	Europe	1977	70.97	1746919	15277.03017
Somalia	Africa	1977	41.974	4353666	1450.992513
South Africa	Africa	1977	55.527	27129932	8028.651439
Spain	Europe	1977	74.39	36439000	13236.92117
Sri Lanka	Asia	1977	65.949	14116836	1348.775651
Sudan	Africa	1977	47.8	17104986	2202.988423
Swaziland	Africa	1977	52.537	551425	3781.410618
Sweden	Europe	1977	75.44	8251648	18855.72521
Switzerland	Europe	1977	75.39	6316424	26982.29052
Syria	Asia	1977	61.195	7932503	3195.484582


Some of you might have noticed that Singapore comes up three times, while every country is only supposed to show up once at most. Upon closer inspection, you can see why this is happening: the `1977` pattern is appearing in the line within the population number, which is not something we want. 

The immediate solution to this is to prevent matches of the year within other numbers. In regex, you can specify that word boundaries must be present before and after the number using `\b`. 

In [35]:
egrep '^(country|S)' gapminder.tsv | egrep '\b1977\b'

Sao Tome and Principe	Africa	1977	58.55	86796	1737.561657
Saudi Arabia	Asia	1977	58.69	8128505	34167.7626
Senegal	Africa	1977	48.879	5260855	1561.769116
Serbia	Europe	1977	70.3	8686367	12980.66956
Sierra Leone	Africa	1977	36.788	3140897	1348.285159
Singapore	Asia	1977	70.795	2325300	11210.08948
Slovak Republic	Europe	1977	70.45	4827803	10922.66404
Slovenia	Europe	1977	70.97	1746919	15277.03017
Somalia	Africa	1977	41.974	4353666	1450.992513
South Africa	Africa	1977	55.527	27129932	8028.651439
Spain	Europe	1977	74.39	36439000	13236.92117
Sri Lanka	Asia	1977	65.949	14116836	1348.775651
Sudan	Africa	1977	47.8	17104986	2202.988423
Swaziland	Africa	1977	52.537	551425	3781.410618
Sweden	Europe	1977	75.44	8251648	18855.72521
Switzerland	Europe	1977	75.39	6316424	26982.29052
Syria	Asia	1977	61.195	7932503	3195.484582


Indeed, this solves our problem, but we lost the header again. To get it back, we need to include the `country` pattern in both commands, which is slightly repetitive. 

**N.B.** Our current solution to filtering for observations made in 1977 is imperfect, because we are filtering on the presence of 1977 anywhere in the line. Technically, if country had a population or GDP per capita of 1977 at some point, this would be included in the output. Later, we will see how we can use awk to apply regex on specific columns. 

In [37]:
egrep '^(country|S)' gapminder.tsv | egrep '(country|\b1977\b)'

country	continent	year	lifeExp	pop	gdpPercap
Sao Tome and Principe	Africa	1977	58.55	86796	1737.561657
Saudi Arabia	Asia	1977	58.69	8128505	34167.7626
Senegal	Africa	1977	48.879	5260855	1561.769116
Serbia	Europe	1977	70.3	8686367	12980.66956
Sierra Leone	Africa	1977	36.788	3140897	1348.285159
Singapore	Asia	1977	70.795	2325300	11210.08948
Slovak Republic	Europe	1977	70.45	4827803	10922.66404
Slovenia	Europe	1977	70.97	1746919	15277.03017
Somalia	Africa	1977	41.974	4353666	1450.992513
South Africa	Africa	1977	55.527	27129932	8028.651439
Spain	Europe	1977	74.39	36439000	13236.92117
Sri Lanka	Asia	1977	65.949	14116836	1348.775651
Sudan	Africa	1977	47.8	17104986	2202.988423
Swaziland	Africa	1977	52.537	551425	3781.410618
Sweden	Europe	1977	75.44	8251648	18855.72521
Switzerland	Europe	1977	75.39	6316424	26982.29052
Syria	Asia	1977	61.195	7932503	3195.484582


Second, we can combine our patterns into one regex. Admittedly, there is no compelling advantage in doing so other than preventing needless commands wherever possible. For this, we need to acknowledge that the year will always be after the country name by some number of characters. We can specify "some numbers of characters" in regex using `.*`. 

In [38]:
egrep '^(country|S).*\b1977\b' gapminder.tsv

Sao Tome and Principe	Africa	1977	58.55	86796	1737.561657
Saudi Arabia	Asia	1977	58.69	8128505	34167.7626
Senegal	Africa	1977	48.879	5260855	1561.769116
Serbia	Europe	1977	70.3	8686367	12980.66956
Sierra Leone	Africa	1977	36.788	3140897	1348.285159
Singapore	Asia	1977	70.795	2325300	11210.08948
Slovak Republic	Europe	1977	70.45	4827803	10922.66404
Slovenia	Europe	1977	70.97	1746919	15277.03017
Somalia	Africa	1977	41.974	4353666	1450.992513
South Africa	Africa	1977	55.527	27129932	8028.651439
Spain	Europe	1977	74.39	36439000	13236.92117
Sri Lanka	Asia	1977	65.949	14116836	1348.775651
Sudan	Africa	1977	47.8	17104986	2202.988423
Swaziland	Africa	1977	52.537	551425	3781.410618
Sweden	Europe	1977	75.44	8251648	18855.72521
Switzerland	Europe	1977	75.39	6316424	26982.29052
Syria	Asia	1977	61.195	7932503	3195.484582


----
### Challenge Question 1

Why is the header missing in output of the above command?

----

Let's create a file with a list of countries of interest for the purposes of this demo. 

In [43]:
echo -e "Canada\nItaly\nAustralia\nUnited States\nEngland\nFrance" > countries.txt
cat countries.txt

Canada
Italy
Australia
United States
England
France


Given this list, we can easily filter the gapminder dataset for observations made for these countries. 

In [45]:
egrep -f countries.txt gapminder.tsv | head

Australia	Oceania	1952	69.12	8691212	10039.59564
Australia	Oceania	1957	70.33	9712569	10949.64959
Australia	Oceania	1962	70.93	10794968	12217.22686
Australia	Oceania	1967	71.1	11872264	14526.12465
Australia	Oceania	1972	71.93	13177000	16788.62948
Australia	Oceania	1977	73.49	14074100	18334.19751
Australia	Oceania	1982	74.74	15184200	19477.00928
Australia	Oceania	1987	76.32	16257249	21888.88903
Australia	Oceania	1992	77.56	17481977	23424.76683
Australia	Oceania	1997	78.83	18565243	26997.93657


----

----

----

## Search-and-replace text using sed

In [40]:
head gapminder.tsv

country	continent	year	lifeExp	pop	gdpPercap
Afghanistan	Asia	1952	28.801	8425333	779.4453145
Afghanistan	Asia	1957	30.332	9240934	820.8530296
Afghanistan	Asia	1962	31.997	10267083	853.10071
Afghanistan	Asia	1967	34.02	11537966	836.1971382
Afghanistan	Asia	1972	36.088	13079460	739.9811058
Afghanistan	Asia	1977	38.438	14880372	786.11336
Afghanistan	Asia	1982	39.854	12881816	978.0114388
Afghanistan	Asia	1987	40.822	13867957	852.3959448
Afghanistan	Asia	1992	41.674	16317921	649.3413952


----

----

----

## Filter and/or process tabular data using awk

In [41]:
head gapminder.tsv

country	continent	year	lifeExp	pop	gdpPercap
Afghanistan	Asia	1952	28.801	8425333	779.4453145
Afghanistan	Asia	1957	30.332	9240934	820.8530296
Afghanistan	Asia	1962	31.997	10267083	853.10071
Afghanistan	Asia	1967	34.02	11537966	836.1971382
Afghanistan	Asia	1972	36.088	13079460	739.9811058
Afghanistan	Asia	1977	38.438	14880372	786.11336
Afghanistan	Asia	1982	39.854	12881816	978.0114388
Afghanistan	Asia	1987	40.822	13867957	852.3959448
Afghanistan	Asia	1992	41.674	16317921	649.3413952


----

----

----

## Solutions to Challenge Questions

### Challenge Question 1

The header is missing from the output because the `.*\b1977\b` in the pattern is restricting that all lines (_i.e._ those starting with `country` or `S`) have a `1977` in it. The solution is to move the `.*\b1977\b` inside the parentheses such that it only applies to lines starting with `S`. 

In [39]:
egrep '^(country|S.*\b1977\b)' gapminder.tsv

country	continent	year	lifeExp	pop	gdpPercap
Sao Tome and Principe	Africa	1977	58.55	86796	1737.561657
Saudi Arabia	Asia	1977	58.69	8128505	34167.7626
Senegal	Africa	1977	48.879	5260855	1561.769116
Serbia	Europe	1977	70.3	8686367	12980.66956
Sierra Leone	Africa	1977	36.788	3140897	1348.285159
Singapore	Asia	1977	70.795	2325300	11210.08948
Slovak Republic	Europe	1977	70.45	4827803	10922.66404
Slovenia	Europe	1977	70.97	1746919	15277.03017
Somalia	Africa	1977	41.974	4353666	1450.992513
South Africa	Africa	1977	55.527	27129932	8028.651439
Spain	Europe	1977	74.39	36439000	13236.92117
Sri Lanka	Asia	1977	65.949	14116836	1348.775651
Sudan	Africa	1977	47.8	17104986	2202.988423
Swaziland	Africa	1977	52.537	551425	3781.410618
Sweden	Europe	1977	75.44	8251648	18855.72521
Switzerland	Europe	1977	75.39	6316424	26982.29052
Syria	Asia	1977	61.195	7932503	3195.484582
